<a href="https://colab.research.google.com/github/ammobam/Study_DeepLearing/blob/main/210818_RNN%2C_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC_Pytorch%EC%9D%B4%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** RNN
- 시계열 예측, 자연어 처리
- 과거의 데이터로 미래를 '예측'하기 위함

- cf) CNN : 이미지 처리
- 작은 패턴으로 분할하여 전체를 자세히 관찰하기 위함

## 1. RNN
- Recurrent Neural Network (순환신경망)

### (1) RNN의 이해
	- TRIANGEL, INTEGRAL
	- 단어를 구성하는 단어는 동일하지만 순서가 다르면 다른 단어가 됨
- 순서를 따르는 패턴을 찾아서 상관관계, 인과관계를 찾고자 RNN이 개발됨
- 일반적인 인공신경망
	- 입력층(입력) > 은닉층(연산) > 은닉층(연산) > 출력층(출력)
- RNN
	- 입력층(입력) > 은닉층(연산) > 은닉층(연산) > 출력층(출력)
	       ┃└─────────────┘            ┃
	       └─────────────────────┘
	- 은닉층에서 연산할 때 이전에 수행한 연산 + 현재연산의 조합으로 연산을 수행함

### (2) RNN의 과정
- timestep = 0
	- 어떤 초기값과 계산을 시작함
	- 은닉층 값을 계산하고 결과값이 나오면 다음 은닉층으로 전송
	- 현재 계산값을 보존하고 있다가
- 다음 계산을 수행하는 timestep=1 시점에서
	- 현재의 입력과 이전에 계산한 값으로 다시 결과를 생성하여 다음 은닉층에 전달함
- 이 과정을 전체 timestep에 지정한 만큼 반복하게 됨

### (3) RNN의 역전파
- BPTT (Back Propagation Through Time)
- t=0, t=3까지 연산을 수행한 경우, 현재 시점까지 전체에 대하여 역전파를 수행해야 함

- 예시
	- (단어 pytorch) 알파벳을 한 개 씩 입력 받아서 다음 알파벳을 예측하는 경우,
		- p 입력 > y 예측 > py를 입력 > t 예측
		- t가 아닌 글자를 예측했다면 잘못된 예측을 수행했으므로 손실을 계산하여 예측함
	- DNN, CNN 구조
		- 이때 y에게만 영향을 줌
	- RNN 구조
		- 이때 y와 그 이전이 입력값인 p에게도 역전파를 수행함


## 2. RNN의 활성화 함수
- tanh (하이퍼볼릭 탄젠트 함수) 이용

## 3. 실습
- 글자 또는 단어를 입력하면 문장을 생성하는 입력기
    - (1) 일반적인 신경망 이용
    - (2) RNN 이용

## 4. RNN의 문제점
- 입력이 커지면 기울기 소실 문제가 발생함
	- 입력이 커지는 경우
	 	- 문장의 길이가 길어지거나
	 	- 예측하기 위해 훈련하는 데이터의 길이가 길어짐
- tanh()
	- tanh는 미분하게 되면 0에서 1 사이의 값이 나옴
	- 이를 여러 번 곱하면 기울기가 0으로 수렴할 가능성이 높음
- 해결책
	- (1) 활성화 함수를 변경함
	- (2) LSTM, GRU 모델 이용
		- LSTM, GRU : 이전 은닉층을 기억하여 사용하는 방식
		- 미분을 이용하지 않고, 이전의 모든 층을 이용함
		- LSTM : 이전 층의 셀 상태를 기억함
		- GRU : 이전층을 단순화하여 RNN와 비슷하게 만들고 LSTM의 성능을 나타내도록 한 것




















## 일반적인 신경망과 RNN 모델의 차이

### 일반적인 신경망

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#### 하이퍼 파라미터
- 학습률
    - 경사하강법에서 보폭에 해당함
    - 전체 최저점을 못 찾는 경우, 학습률 줄이기
    - 과대적합의 경우, 학습률 늘리기
    - 학습 시간을 단축하고자 하는 경우, 학습률 늘리기
- 반복횟수
    - 모델 생성을 위한 훈련 횟수

In [ ]:
# 하이퍼 파라미터 설정
n_hidden = 35    # 은닉
lr = 0.01        # 학습률
epochs = 1000    # 반복횟수

#### 샘플데이터 / 토큰 생성

In [ ]:
# 샘플 데이터와 분류를 위한 토큰 생성

# 샘플 데이터
string = 'hello, pytorch. How long can a RNN cell remember? show me yout limit!'

# 문장에 포함된 글자들. 0과 1은 시작과 종료를 위한 문장
## 스페이스도 잊지 말고 넣어주자.
chars = 'abcdefghijklmnopqrstuvwxyz ?!,.01'

# 글자 개수 저장
char_list = [i for i in chars]
n_letters = len(char_list)

#### 원핫 인코딩

In [ ]:
# 원핫 인코딩을 위한 함수
def string_to_onehot(string):

    # 입력받은 문장을 소문자로 변환
    string = string.lower()

    # 시작과 끝 토큰 생성
    start = np.zeros(shape=n_letters, dtype=int)
    end = np.zeros(shape=n_letters, dtype=int)

    # 각 데이터에 start, end를 나타내기 위한 작업
    ## [start] 데이터 데이터 데이터 [end]
    ## 그래서 맨 처음 start 지점에서 iterator 돌릴 때 next()로 첫 데이터를 불러와야 함
    start[-2] = 1
       # BOW : Begin of Word
    end[-1] = 1

    # 입력된 문자 순회
    for i in string:
        # 글자의 인덱스 찾기
        idx = char_list.index(i)
        #print(i)
        
        # 각 글자를 표현하기 위한 배열 생성
        ## 0으로 채워진 글자 길이만크의 배열 생성
        zero = np.zeros(shape=n_letters, dtype=int)
        ## 글자 있는 부분은 1로 채움
        zero[idx] = 1
        ## 시작과 끝을 이음
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [ ]:
# 함수 실행
print(len(string))
print(string_to_onehot(string).shape) # start, end 더하기
print(string_to_onehot(string))

print(string_to_onehot('hi').shape) # start, end 더하기
print(string_to_onehot('hi'))

69
(71, 33)
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
(4, 33)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [ ]:
# 원핫인코딩 된 문자열을 복원하는 함수
def onehot_to_word(onehot_1):
    # 원핫인코딩 된 숫자들을 텐서로 변환
    #onehot = torch.Tensor(onehot_1)
    onehot = torch.Tensor.numpy(onehot_1)
    # 가장 큰 인덱스에 해당하는 글자로 리스트 생성
    return char_list[onehot.argmax()]

In [ ]:
# 함수 수행
#onehot_to_word(torch.Tensor(string_to_onehot('hi')))
onehot_to_word((string_to_onehot('hi')))

TypeError: ignored

#### RNN 모델 설계

In [ ]:
# 모델 설계
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        # 인스턴스 변수(속성, 프로퍼티)를 생성해서 값을 대입
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # RNN은 이전 층의 결과값을 가져와야 함! (입력, 출력)
        ## 입력의 크기를 설정할 때 hidden_size에 더함
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        # RNN은 활성화 함수로 tanh 함수를 사용함
        # act_fn : activation function
        self.act_fn = nn.Tanh()

    def forward(self, input, hidden):
        # 현재입력과 이전출력을 붙이기
        combined = torch.cat((input, hidden), 1)
        # 역전파를 위한 hidden 상태의 업데이트
        hidden = self.act_fn(self.i2h(combined))
        # 출력 만들기
        #output = self.i2o(hidden) # 이게 아닌가? 아님!!
        output = self.i2o(combined) # 여기서 output은 최종 출력층이 아니라 각 층의 출력부를 말함

        return output, hidden # 현재 셀의 출력과, 이전 셀의 출력을 같이 가져감! RNN

    # 맨 처음에는 은닉층이 없음
    # 맨 처음의 hidden state를 만들어주는 함수
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
# 모델 생성
rnn = RNN(n_letters, n_hidden, n_letters)

#### 손실함수, 최적화함수(optimizer) 설정

In [ ]:
# 손실함수와 최적화함수 생성
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

#### 모델 훈련

In [ ]:
# 모델 훈련용 데이터 만들기

# 문자열을 원핫벡터로 만들고 텐서로 변경
# 데이터 타입 변경
one_hot = torch.from_numpy( string_to_onehot(string) ).type(torch.FloatTensor)
#one_hot = torch.from_numpy( string_to_onehot(string) ).as_type(torch.FloatTensor()) # 원래코드 ??
    # type() : 함수를 변환함
    # type_as() : 데이터를 변환함
    # 에러가 나서 type으로 바꿔줌
    # https://pytorch.org/docs/stable/generated/torch.Tensor.type_as.html

# 확인
print(type(one_hot))
print(one_hot.shape)
print(one_hot.size())
print(type(one_hot.shape))
print(type(one_hot.size()))

<class 'torch.Tensor'>
torch.Size([71, 33])
torch.Size([71, 33])
<class 'torch.Size'>
<class 'torch.Size'>


In [ ]:
# 모델 훈련
for i in range(epochs):
    # 최적화 함수 초기화
    optimizer.zero_grad()     # zero_grad: optimizer의 시작 지점을 0으로 초기화해줌
                              # Sets gradients of all model parameters to zero.
    # 첫번째 은닉층 초기화
    hidden = rnn.init_hidden()
    
    # 전체 손실 저장할 변수 생성
    total_loss = 0

    # 훈련
    for j in range(one_hot.size()[0]-1): # 70번..... 왜 -1 해주지? -2가 아니라??
        # 입력은 앞글자
        input_ = one_hot[j:j+1, :]
        # 목표는 다음 글자
        target = one_hot[j+1]

        output, hidden = rnn.forward(input_, hidden)

        loss = loss_func(output.view(-1), target.view(-1)) # .view??
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(3.1007, grad_fn=<AddBackward0>)
tensor(1.2763, grad_fn=<AddBackward0>)
tensor(0.7439, grad_fn=<AddBackward0>)
tensor(0.4429, grad_fn=<AddBackward0>)
tensor(0.3041, grad_fn=<AddBackward0>)
tensor(0.2229, grad_fn=<AddBackward0>)
tensor(0.1578, grad_fn=<AddBackward0>)
tensor(0.1198, grad_fn=<AddBackward0>)
tensor(0.1130, grad_fn=<AddBackward0>)
tensor(0.0825, grad_fn=<AddBackward0>)
tensor(0.0686, grad_fn=<AddBackward0>)
tensor(0.0581, grad_fn=<AddBackward0>)
tensor(0.0697, grad_fn=<AddBackward0>)
tensor(0.0467, grad_fn=<AddBackward0>)
tensor(0.0417, grad_fn=<AddBackward0>)
tensor(0.0380, grad_fn=<AddBackward0>)
tensor(0.0352, grad_fn=<AddBackward0>)
tensor(0.0357, grad_fn=<AddBackward0>)
tensor(0.0330, grad_fn=<AddBackward0>)
tensor(0.0299, grad_fn=<AddBackward0>)
tensor(0.0277, grad_fn=<AddBackward0>)
tensor(0.0337, grad_fn=<AddBackward0>)
tensor(0.0268, grad_fn=<AddBackward0>)
tensor(0.0246, grad_fn=<AddBackward0>)
tensor(0.0230, grad_fn=<AddBackward0>)
tensor(0.0316, grad_fn=<A

In [ ]:
help(np.ndarray.view) ###########################################################################
# a.view([dtype][, type])
# New view of array with the same data.

Help on method_descriptor:

view(...)
    a.view([dtype][, type])
    
    New view of array with the same data.
    
    .. note::
        Passing None for ``dtype`` is different from omitting the parameter,
        since the former invokes ``dtype(None)`` which is an alias for
        ``dtype('float_')``.
    
    Parameters
    ----------
    dtype : data-type or ndarray sub-class, optional
        Data-type descriptor of the returned view, e.g., float32 or int16.
        Omitting it results in the view having the same data-type as `a`.
        This argument can also be specified as an ndarray sub-class, which
        then specifies the type of the returned object (this is equivalent to
        setting the ``type`` parameter).
    type : Python type, optional
        Type of the returned view, e.g., ndarray or matrix.  Again, omission
        of the parameter results in type preservation.
    
    Notes
    -----
    ``a.view()`` is used two different ways:
    
    ``a.view(some_dt

#### ■ 슬라이싱과 copy에 대하여

In [ ]:
# 왜 배열 슬라이싱을 할까 ?
print(one_hot[0:0+1, :] == one_hot[0, :])  # 값은 같지만
print(one_hot[0:0+1, :] is one_hot[0, :])  # 다른 객체임
## one_hot[0:0+1, :]은 copy()로서 원본 객체에 영향을 주지 않음
## one_hot[0, :]은 참조 복사로서 변경시 원본 객체에 영향을 줌(같이 수정됨)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True]])
False


# ** 자연어 처리
- NLP(Natural Language Processing)
- Text 데이터를 분석하고 모델링하는 분야
- 자연어 이해(Natural Language Understanding) + 자연어 생성(Natural Language Generation) 영역으로 구분하기도 함
- 자연어 이해 : Text의 의미를 파악하는 것
- 자연어 생성 : 주어진 의미에 대한 자연스러운 Text를 만들어내는 것

## 1. 분야
- 1) 감성 분석
- 2) 요약
	- 문장의 의미를 파악하여 새로운 문장을 만들어내야 함
- 3) 기계 번역
- 4) 질의 응답
	- 현업에서는 API를 많이 이용함
	- 한국어 데이터셋 : korQuAD
- 5) Pos Tagging
	- 형태소 분석, 품사 예측
- 6) 챗봇
- 7) 문장 간의 논리적인 관계에 대한 분류 모델
	- GAN
	- 창작의 영역(소설, 이미지, 음악)에서 많이 쓰임
- 8) Image Captioning
	- 이미지를 설명하는 문장 붙임

## 2. 자연어처리 참고 링크
- 참고링크 : http://nlpprogress.com
- 참고링크 : https://paperswithcode.com/area/natural-language-processing
- 자연어 처리 리더보드
	- 한국 : korquad.github.io
	- 세계 : https://rajpurkar.github.io/SQuAD-explorer/


## 자연어 전처리
- 숫자로 표현하는 방법(텐서, 배열)
- 1) 문장을 숫자로 표현하기 위한 순서
	- Text Segmentation
	 	- 문장을 의미있는 단위로 나눔
	- Representation
	 	- 의미있는 부분을 숫자로 변환
- 2) 데이터를 최소 단위로 분할하기
	- str 클래스의 split을 이용한 공백 단위 분할
	- 글자 단위로 나누기
		- 파이썬은 str이 글자의 list임
		- list(문자열) : 글자 단위로 list 생성

### 샘플 문장

In [ ]:
# 샘플 문장 데이터
sample1 = '다른 사람의 이야기를 듣는 건 즐겁다.'
sample2 = '가끔 실제가 아닌 가상 인물의 이야기를 듣는 것도 좋다.'
sample3 = '하지만 허구의 이야기는 창작자의 경험에 기반한 것으로 패턴을 알고 나면 지겹다.'
sample4 = '요즘은 극을 보고 싶은데 이미 다 봤던 극이거나 관심 없는 극이라서 볼 게 없다.'
sample5 = '극작이나 연출가가 본업 실력으로 떳으면 좋겠다.'
sample6 = '소재만 자극적이고 전달하는 메세지도 또렷하지 않은 극은 재미 없고 극작의 마인드가 낡은 경우 불쾌하기까지 하다.'

### 데이터 최소단위로 분할하기

In [ ]:
# 단어 단위로 분할
# str의 split 이용
## 정규식을 기준으로 문자열 분할하여 list 생성
## 정규식을 설정하지 않으면 공백 단위로 분할함
print(sample1.split())

['다른', '사람의', '이야기를', '듣는', '건', '즐겁다.']


In [ ]:
# 글자 단위로 분할
# 파이썬의 str은 문자열의 list임을 이용
print(list(sample2))

['가', '끔', ' ', '실', '제', '가', ' ', '아', '닌', ' ', '가', '상', ' ', '인', '물', '의', ' ', '이', '야', '기', '를', ' ', '듣', '는', ' ', '것', '도', ' ', '좋', '다', '.']


### 분할한 데이터를 수치화하기
- 중복된 단어는 1번만 수행

In [ ]:
# 공백을 기준으로 분할한 단어를 수치화하여 딕셔너리에 저장

# 데이터를 저장할 딕셔너리 생성
token2idx = {}
index = 0

# 문장들에 대하여
for sentence in [sample1, sample2, sample3, sample4, sample5, sample6]:
    # 각 문장을 단어 단위로 분할
    tokens = sentence.split()
    # 분할한 문장을 단어 단위로 순회함
    for token in tokens:
        # dict에 존재하지 않으면 추가함 - key는 중복되지 않으므로 ! 중복 단어는 1번만 수행됨
        # 인덱스 값을 1씩 증가시켜서 저장
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1

print(token2idx)

{'다른': 0, '사람의': 1, '이야기를': 2, '듣는': 3, '건': 4, '즐겁다.': 5, '가끔': 6, '실제가': 7, '아닌': 8, '가상': 9, '인물의': 10, '것도': 11, '좋다.': 12, '하지만': 13, '허구의': 14, '이야기는': 15, '창작자의': 16, '경험에': 17, '기반한': 18, '것으로': 19, '패턴을': 20, '알고': 21, '나면': 22, '지겹다.': 23, '요즘은': 24, '극을': 25, '보고': 26, '싶은데': 27, '이미': 28, '다': 29, '봤던': 30, '극이거나': 31, '관심': 32, '없는': 33, '극이라서': 34, '볼': 35, '게': 36, '없다.': 37, '극작이나': 38, '연출가가': 39, '본업': 40, '실력으로': 41, '떳으면': 42, '좋겠다.': 43, '소재만': 44, '자극적이고': 45, '전달하는': 46, '메세지도': 47, '또렷하지': 48, '않은': 49, '극은': 50, '재미': 51, '없고': 52, '극작의': 53, '마인드가': 54, '낡은': 55, '경우': 56, '불쾌하기까지': 57, '하다.': 58}


In [ ]:
# 문장 안의 token을 key로 이용하여 token2idx 딕셔너리에서 숫자(index)를 꺼내옴
def indexed_sentence(sentence):
    return [token2idx[token] for token in sentence]
    # token은 sentence 안에 있는 경우만 수행하기 때문에, 새로운 단어가 오면 오류 남

In [ ]:
# 함수 수행
result = indexed_sentence(sample1.split())
print(result)

result = indexed_sentence(sample2.split())
print(result)
print()

print("모든 샘플에 대해 인덱스 꺼내보기")
for i, sample in enumerate([sample1, sample2, sample3, sample4, sample5, sample6]):
    result = indexed_sentence(sample.split())
    print("sample", i+1, ":", result)

[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 2, 3, 11, 12]

모든 샘플에 대해 인덱스 꺼내보기
sample 1 : [0, 1, 2, 3, 4, 5]
sample 2 : [6, 7, 8, 9, 10, 2, 3, 11, 12]
sample 3 : [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
sample 4 : [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
sample 5 : [38, 39, 40, 41, 42, 43]
sample 6 : [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]


In [ ]:
# 문제점
# 위 딕셔너리에 없는 문자가 오면 작동하지 않음
#result = indexed_sentence("이번에도 호프가 올까?".split())
#print(result)

### OOV를 위한 token 만들기
- 딕셔너리에 없는 token이 오면 수행하지 않음
- 해결방법 : OOV를 위한 token을 별도로 만들기
	- OOV : Out of Vocabulary


In [ ]:
# OOV를 위한 별도의 토큰 추가

## 아는 단어가 나오면 값을 1 추가
token2idx = {t:i+1 for t, i in token2idx.items()}
    # token2idx.items() : key, val 튜플

## 모르는 단어가 나올 때 불러올 키, 값 생성
token2idx['<unknown>'] = 0

## 함수 수정
def indexed_sentence(sentence):
    return [token2idx.get(token, token2idx['<unknown>']) for token in sentence]
    # 딕셔너리.get(입력한 키, 해당 키가 없을 때 리턴할 디폴트 키)

In [ ]:
help(dict.get)

Help on method_descriptor:

get(self, key, default=None, /)
    Return the value for key if key is in the dictionary, else default.



In [ ]:
# 이젠 새로운 단어가 오면 0값을 리턴함
result = indexed_sentence("이번에도 호프가 올까?".split())
print(result)

[0, 0, 0]


## Corpus
- Corpus : token을 만들기 위해서 모아 놓은 문장의 모음
- 크롤링하여 생성
- 이미 만들어진 corpus 사용
	- 영어 corpus : https://www.english-corpora.org
	- 구글의 corpus는 740GB가 넘음


## BPE 알고리즘 구현
- Byte Pair Encoding
- 1) 한국어의 단위
	- 음운 < 음절 < 형태소 < 단어 < 어절 < 문장
		- 음운 : 소리의 단위
		- 음절 : 글자
		- 형태소 : 의미를 가진 최소 단위
		- 단어 : 최소의 자립 형식. 일반적으로 공백 기준으로 분할. 조사는 한 단어임을 유의.
		- 어절V : 문장을 이루는 마디. 띄어쓰기의 단위.
	- 영어는 단어와 어절이 동일하여 띄어쓰기로 분할
	- 한국어는 조사를 뗀 단어가 필요함
- 2) '글자' 단위로 사전을 생성하는 경우
	- 더 적은 사전으로 여러 문장 처리 가능함
	- OOV 문제가 해결됨
	- 하지만 글자는 그 자체로 의미를 갖지 않는 경우가 대부분임
- 3) n_gram Tokenization
	- 글자, 글자 결합에 초점을 둠
	- uni_gram : 하나의 글자만 이용하는 경우
	- b i_gram : 2개의 글자를 이용하는 경우
	- 문제점
	 	- 이 방법은 단어 사전이 너무 커짐
		- 한 번만 등장하고 다시는 등장하지 않는 단어도 사전에 등록해야 하기 때문임
- 4) BPE 이용
	- n_gram 중에서 사용이 될 것 같은 단어만 모은 것
	- 반복적으로 나오는 데이터의 패턴을 치환하는 방식을 사용함
	- 데이터를 효율적으로 저장하기 위한 압축 알고리즘

- 예시
	- abbcabcab
	- step 1) 가장 많이 등장하는 2개짜리 패턴을 찾아서 X로 치환
		- ab가 제일 많이 등장
		- XbcXcX
	- step 2) 반복 :  패턴이 없어질 때까지/혹은 일정 횟수 이상
		- Y = cX
		- XbYY


In [ ]:
import re, collections

# 단어의 등장 횟수를 dict에 저장하는 함수  ???????????
# vocab은 딕셔너리
def get_stats(vocab):
    # 정수 딕셔너리 생성
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):    # <\w>를 빼고 세려고 -1 함
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

- help(collections.defaultdict(int))
```
class defaultdict(builtins.dict)
 |  defaultdict(default_factory[, ...]) --> dict with default factory
 |  
 |  The default factory is called without arguments to produce
 |  a new value when a key is not present, in __getitem__ only.
 |  A defaultdict compares equal to a dict with the same items.
 |  All remaining arguments are treated the same as if they were
 |  passed to the dict constructor, including keyword arguments.
 ```

In [ ]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    #p = re.compile(r'(?<!\S)' + bigram + r'(?<!\S)')
    # <가 메타문자에서 뭐길래..?
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out
# re : 정규표현식 메타문자
# https://developer.mozilla.org/ko/docs/Web/JavaScript/Guide/Regular_Expressions

In [ ]:
# 테스트할 단어 생성
vocab = ({'l o w </w>':5,        # 키 : l o w </w>, 값 : 5
          'l o w e r </w>':2,
          'n e w e s t </w>':6,
          'w i d e s t </w>':3})

# 수행횟수 설정
num_merges = 10
for i in range(num_merges):
    # 연속된 2글자의 등장 횟수 셈
    pairs = get_stats(vocab)
    # 가장 많이 등장한 페어를 찾음
    best = max(pairs, key = pairs.get)
    # 가잔 많이 등장한 페어의 공백 제거
    vocab = merge_vocab(best, vocab)
    print(f'Step {i+1}')
    print(best)
    print(vocab)
    print('\n')

#get_stats(vocab)

Step 1
('e', 's')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


Step 2
('es', 't')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


Step 3
('est', '</w>')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 4
('l', 'o')
{'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 5
('lo', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 6
('n', 'e')
{'low </w>': 5, 'low e r </w>': 2, 'ne w est</w>': 6, 'w i d est</w>': 3}


Step 7
('ne', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}


Step 8
('new', 'est</w>')
{'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 9
('low', '</w>')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 10
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}




## 원핫인코딩 구현

In [41]:
# 단어사전 만드는 데이터
s1 = "이게 뭐지 이게 뭘까 자고 일어나니 세상이 바뀐 것처럼"
s2 = "뮤지컬 보고싶다. 본 적 없고 연출 좋고 노래 좋은 뮤지컬"
s3 = "무신경한 연출과 어디서 들어본 것 같은 대사는 이제 지겨워"
s4 = "배우가 좋으면 그 순간은 좋겠지. 곱씹을수록 별로라 그렇지"
s5 = "대극장 뮤지컬이 보고싶은 걸까?"
s6 = "마틸다랑 라이언킹은 연출도 내용도 좋았다."

In [42]:
# 딕셔너리 생성
token2idx = {}
index = 0
for sentence in [s1, s2, s3, s4, s5, s6]:
    # 각 문장을 공백 기준으로 나누기
    tokens = sentence.split()
    for token in tokens:
        # 각 띄어쓰기 단어
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1
print(token2idx)

{'이게': 0, '뭐지': 1, '뭘까': 2, '자고': 3, '일어나니': 4, '세상이': 5, '바뀐': 6, '것처럼': 7, '뮤지컬': 8, '보고싶다.': 9, '본': 10, '적': 11, '없고': 12, '연출': 13, '좋고': 14, '노래': 15, '좋은': 16, '무신경한': 17, '연출과': 18, '어디서': 19, '들어본': 20, '것': 21, '같은': 22, '대사는': 23, '이제': 24, '지겨워': 25, '배우가': 26, '좋으면': 27, '그': 28, '순간은': 29, '좋겠지.': 30, '곱씹을수록': 31, '별로라': 32, '그렇지': 33, '대극장': 34, '뮤지컬이': 35, '보고싶은': 36, '걸까?': 37, '마틸다랑': 38, '라이언킹은': 39, '연출도': 40, '내용도': 41, '좋았다.': 42}


In [43]:
# 샘플데이터에 등장한 단어의 개수
V = len(token2idx)

# 단어가 등장한 idx가 아니면 0을 채워넣음.
# 배열, ㅎㅎ
token2vec = [(   [0 if i != idx else 1 for i in range(V)]   , idx, token)    for token, idx in token2idx.items()]

for x in token2vec:
    print('\t'.join([str(y) for y in x]))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	0	이게
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	1	뭐지
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	2	뭘까
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	3	자고
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	4	일어나니
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	5	세상이
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	6	바뀐
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [48]:
# 각 샘플 문장을 원핫 인코딩 하기
import numpy as np
for sentence in [s1, s2, s3]:
    onehot_s = []
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) != None:
            vector = np.zeros((1, V))
            vector[:, token2idx[token]] = 1
            onehot_s.append(vector)
        else:
            print("unknown")
    print(f"{sentence}:")
    print((np.concatenate(onehot_s, axis=0)).shape)
    print(np.concatenate(onehot_s, axis=0))    # ndarray    # shape: (n, 43)
    print()

이게 뭐지 이게 뭘까 자고 일어나니 세상이 바뀐 것처럼:
(9, 43)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.

## Pre-Trained Word Embedding 사용하기

### google의 Sentencepiece 사용

In [1]:
# 라이브러리 설치
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.3 MB/s 


In [2]:
# 모델 다운로드
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
# !wget -P /경로/ 다운받을파일링크


--2021-08-18 08:22:57--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.02s   

2021-08-18 08:22:57 (13.0 MB/s) - ‘botchan.txt’ saved [278779/278779]



In [8]:
import sentencepiece as spm

# 모델에 데이터 설정
spm.SentencePieceTrainer.train('--input=/content/botchan.txt --model_prefix=m --vocab_size=2000')
                                                    # 모델 경로    # 모델 이름 설정?
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [10]:
# 단어 단위 토큰화
## 토큰 리턴
print(sp.encode_as_pieces('New York'))
## 아이디 리턴
print(sp.encode_as_ids('New York'))

['▁New', '▁Y', 'or', 'k']
[1437, 867, 105, 94]


In [11]:
# 복원
## 토큰을 이용하여 복원
print(sp.decode_pieces(['▁New', '▁Y', 'or', 'k']))
## 아이디를 이용하여 복원
print(sp.decode_ids([1437, 867, 105, 94]))

New York
New York


### BERT 이용
- Transformers 설치
- BERT의 버전
    - 기본 버전 : 'bert-base-uncased'
    - 다국어 버전 : 'bert-base-multilingual-uncased'

In [13]:
# BERT 사용을 위한 Transformers 설치
! pip install transformers

     |████████████████████████████████| 2.6 MB 9.4 MB/s 
     |████████████████████████████████| 3.3 MB 54.5 MB/s 
     |████████████████████████████████| 895 kB 62.6 MB/s 
     |████████████████████████████████| 636 kB 58.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [22]:
from transformers import BertTokenizer

# 기본 버전 가져오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print(len(tokenizer.vocab))    # 약 30522개의 단어가 있음

# 토큰화 테스트
print(tokenizer.tokenize('defalult version only supports English'))

30522
['def', '##al', '##ult', 'version', 'only', 'supports', 'english']


In [24]:
# 다국어 버전 가져오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

print(len(tokenizer.vocab))    # 약 30522개의 단어가 있음

# 토큰화 테스트
print(tokenizer.tokenize('다국어 version은 한국어 지원함. 근데 영어로 섞여도 잘 될까?'))
## 고작 '지'를 몰라서 ㅈ + ㅣ로 나눈다고..?

105879
['다', '##국', '##어', 'version', '##은', '한국', '##어', 'ᄌ', '##ᅵ', '##원', '##함', '.', '그', '##ᆫ', '##데', '영어', '##로', 'ᄉ', '##ᅥ', '##ᆩ', '##여', '##도', '잘', '될', '##ᄁ', '##ᅡ', '?']
